Designing a T3:
- Get a lightcurve
- Complement with a T2 result (CatalogMatch here)
- Assemble to T3 input.
- Create T3 like functionality.

In [ ]:
import requests, os
from astropy.time import Time
from ampel.ztf.t0.load.ZTFArchiveAlertLoader import ZTFArchiveAlertLoader
from ampel.ztf.alert.ZiAlertSupplier import ZiAlertSupplier
from ampel.ztf.ingest.ZiDataPointShaper import ZiDataPointShaper
from ampel.log.AmpelLogger import AmpelLogger
from ampel_notebook_utils import alert2dps

In [ ]:
# This is the archive token which can be obtained from https://ampel.zeuthen.desy.de/live/dashboard/tokens
# In order to retrieve ZTF partnership alerts your token needs to have the appropriate access
token = os.environ["ARCHIVE_TOKEN"]   # I have mine stored
header = {"Authorization": "bearer "+token}

##### 1. Obtaining a stream of alerts 
This demonstration will be carried out using a pre-selected set of alerts taken from one ZTF field, and with a set of minimal properties.


In [ ]:
endpoint = 'https://ampel.zeuthen.desy.de/api/ztf/archive/v3/streams/from_query?programid=1'

In [ ]:
# Time window to search archive 
#jdend = Time.now().jd
jdend = 2459800
time_window = 2.

In [ ]:
# Alert query aimed at finding new objects (few detections) in a narrow time-span.
query = {
  "jd": {
    "$gt": jdend - time_window,
    "$lt": jdend + 0.1,
  },
  "candidate": {
    "drb": {
      "$gt": 0.995
    },
    "ndethist": {
      "$gt": 1,
      "$lt": 5,
    },
    "isdiffpos": {"$in": ["t", "1"]},
  }
}

In [ ]:
response = requests.post(endpoint, headers=header, json=query )

if not response.ok:
    print( 'Query creation failed.')
resume_token = response.json()['resume_token']


In [ ]:
# The loader config contains the resume_token as stream identifier
config = {'archive':"https://ampel.zeuthen.desy.de/api/ztf/archive/v3", 
          "stream":resume_token}

Configure and initialize a unit designed for catalog matching.

In [ ]:
from ampel.ztf.t2.T2CatalogMatch import T2CatalogMatch

In [ ]:
lenscatconfig = {
        'NEDz' : {
            'use' : 'catsHTM',
            'rs_arcsec' : 10.0,
            'keys_to_append' : ['ObjType', 'Velocity', 'z'],
        },
        'LSPhotoZZou' : {
            'use' : 'extcats',
            'rs_arcsec' : 10.0,
            'keys_to_append' : ['photoz','ra','dec','e_photoz','specz','_6','logMassBest','logMassInf','logMassSup'],
            'pre_filter' : None,
            'post_filter' : None,
            'all': False,
        },
        'masterlens' : {
            'use' : 'extcats',
            'rs_arcsec' : 10.0,
            'keys_to_append' : ['ra_degrees','dec_degrees','z_lens','z_source',
                                'radius','system_name','comment'],
            'pre_filter' : None,
            'post_filter' : None,
            'all': False,
        },

    }

In [ ]:
# Initalize
logger = AmpelLogger.get_logger()

In [ ]:
t2cat = T2CatalogMatch( catalogs = lenscatconfig,
    resource = {"ampel-ztf/catalogmatch":'https://ampel.zeuthen.desy.de/api/catalogmatch/'},
    logger=logger)

In [ ]:
shaper = ZiDataPointShaper(logger=AmpelLogger.get_logger())

In [ ]:
alert_matches = []

In [ ]:
try:
    alertloader = ZTFArchiveAlertLoader(**config)
    for k, alert in enumerate(alertloader.get_alerts()):
        
        # Modify alert so the shaper understands
        alert['candidate']['candid'] = alert['candid']
        dps = alert2dps(alert,shaper=shaper)
        t2cat_out = t2cat.process(dps[0])
        if t2cat_out['masterlens'] is not None:
            print('Found match to masterlens catalog')
            alert_matches.append({'alert':alert, 'catalog_match':t2cat_out})
        elif t2cat_out['NEDz'] is not None:
            print('Found match to NED z catalog')
            alert_matches.append({'alert':alert, 'catalog_match':t2cat_out})
            
        if k%500==0:
            print('Alert counter:', k)
            
except requests.exceptions.HTTPError as e:
    status_code = e.response.status_code
    if status_code==423:
        print('HTTP error {}: likely caused by server staging process. Wait and try again.'.format(status_code) )
    else:
        raise e

Alerts and the catalog matches that were detected can now be inspected:

In [ ]:
print('Found {} alerts with coordinates matching NED or masterlens in the query.'.format(len(alert_matches)))

In [ ]:
alert_matches[0]['catalog_match']

In [ ]:
import voeventparse as vp

In [ ]:
with open('/home/jnordin/Downloads/voevent.xml','rb') as f:
    v = vp.load(f)


In [ ]:
v.Why.Inference.attrib